<a href="https://colab.research.google.com/github/sikandarmir/Object-Detection-/blob/main/Deep_Learning_Based_Object_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Object Detection:
Object detection is a computer vision technique that allows us to identify and locate objects in an image or video.

**Deep Learning Based Object Detection**
1.Architecture : Mobilenet Based Single shot Multi-Box(SSD)

2:Framework : Tensorflow


**Single Shot MultiBox Detector(SSD)**

Link:https://www.youtube.com/watch?v=UwyzEejuGWk&ab_channel=CodeWithAarohi


*   Single Shot: this means that the tasks of object localization and classification are done in a single forward pass of the network
*   MultiBox: this is the name of a technique for bounding box regression developed by Szegedy et al. (we will briefly cover it shortly)

*   The network is an object detector that also classifies those detected objects.

**YOLO**
YOLO is an abbreviation for the term ‘You Only Look Once’. This is an algorithm that detects and recognizes various objects in a picture (in real-time). Object detection in YOLO is done as a regression problem and provides the class probabilities of the detected images.

YOLO algorithm employs convolutional neural networks (CNN) to detect objects in real-time. As the name suggests, the algorithm requires only a single forward propagation through a neural network to detect objects.

Link:https://www.youtube.com/watch?v=ag3DLKsl2vk&ab_channel=codebasics
In Yolo algorthuim the Image is X Train anad Y Train will be vector of size 7.and the out put it also give the vector of size 7 to tell that this is a particular object or not.  
To Implement The Yolo Algorthuim follow the Follwing link instruction
Link:https://github.com/AlexeyAB/darknet#how-to-compile-on-windows-using-cmake





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!curl -O http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!curl -O https://drive.google.com/file/d/121-kCXaOHOkJE_Kf5lKcJvC_5q1fYb_q/view

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0^C
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0^C


In [ ]:
#Unzip .tar file Extension
import tarfile
my_tar = tarfile.open('/content/VOCtrainval_06-Nov-2007.tar')
my_tar.extractall('/content/drive/MyDrive/Data_set') # specify which folder to extract to
my_tar.close()

In [ ]:
# !pip install adam
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow
import keras
# from keras.optimizers import adam
# # from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

# from models.keras_ssd300 import ssd_300

# from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder

# from data_generator.object_detection_2d_data_generator import DataGenerator
# from data_generator.object_detection_2d_geometric_ops import Resize
# from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
# from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

In [ ]:
img_height = 300 
img_width = 300 
img_channels = 3 
mean_color = [123, 117, 104] 
swap_channels = [2, 1, 0] #BGR instead of RGB
n_classes = 20 
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] 
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] 
two_boxes_for_ar1 = True #used in anchorboxes.py. used for next_scale 
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

In [ ]:

train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)


# Parse the image and label lists for the training and validation datasets. This can take a while.
VOC_2007_images_dir      = '/content/drive/MyDrive/Data_set/VOCdevkit/VOC2007/JPEGImages/'

VOC_2007_annotations_dir      = '/content/drive/MyDrive/Data_set/VOCdevkit/VOC2007/Annotations/'

# The paths to the image sets.
VOC_2007_train_image_set_filename    = '/content/drive/MyDrive/Data_set/VOCdevkit/VOC2007/ImageSets/Main/train.txt'
VOC_2007_val_image_set_filename      = '/content/drive/MyDrive/Data_set/VOCdevkit/VOC2007/ImageSets/Main/val.txt'

VOC_2007_trainval_image_set_filename = '/content/drive/MyDrive/Data_set/VOCdevkit/VOC2007/ImageSets/Main/trainval.txt'
VOC_2007_test_image_set_filename     = '/content/drive/MyDrive/Data_set/VOCdevkit/VOC2007/ImageSets/Main/test.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

train_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                        image_set_filenames=[VOC_2007_trainval_image_set_filename],
                        annotations_dirs=[VOC_2007_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)

val_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                      image_set_filenames=[VOC_2007_test_image_set_filename],
                      annotations_dirs=[VOC_2007_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=True,
                      ret=False)

NameError: ignored

In [ ]:
batch_size = 16

In [ ]:

# Set the image transformations for pre-processing and data augmentation options.

# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

In [ ]:
model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)

In [ ]:
# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'filenames',
                                                  'inverse_transform',
                                                  'original_images',
                                                  'original_labels'},
                                         keep_images_without_gt=False)

In [ ]:
# number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset: ", train_dataset_size)
print("Number of images in the validation dataset:  ", val_dataset_size)

In [ ]:
# 2: Generate samples.

batch_images, batch_filenames, batch_inverse_transforms, batch_original_images, batch_original_labels = next(train_generator)

i = 0 # Which batch item to look at

print("Image:", batch_filenames[i])

print("Ground truth boxes:\n")
print(np.array(batch_original_labels[i]))

In [ ]:
# 5: Draw the predicted boxes onto the image
import matplotlib.pyplot as plt

# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, n_classes+1)).tolist()
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

plt.figure(figsize=(20,12))
plt.imshow(batch_original_images[i])

current_axis = plt.gca()

#plt.plot()

#plotting groundtrithbox
for box in batch_original_labels[i]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    label = '{}'.format(classes[int(box[0])])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})